Let $C = \{c_1, c_2, ..., c_n\}$ be the comments and $X_i$ = $x_i^j , j=1,2,\cdots,m$, where $x_i^j$ the annotation of annotator j for comment i.


Let $X^{\rho}_i$ the annotations for $c_i$ which belong in a partition $\rho \in \Rho$. Each $\rho$ in this case is a factor of feature $\Rho$

Then we can define $aposteriori(c_i) = max_{\rho} \{ndfu(X_i) - ndfu(X_i^ \rho)\}$
and $aposteriori(C) = Wilcoxon(aposteriori(c_i), 0, alternative="greater")$

In [25]:
%load_ext autoreload 
%autoreload 2
import pandas as pd
import numpy as np
from pathlib import Path

from ndfu import ndfu
import aposteriori

import re
import hashlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
df = pd.read_pickle("attitudes_embedded.csv")
df

tweet               toyou  \
0    " you ugly bitch!!!"- that seems to be my real...        [2, 1, 1, 2]   
1    """I am happy to report that drowning does kil...  [4, 2, 1, 2, 1, 1]   
2    """Let them praise the name of the LORD: for h...  [1, 1, 1, 1, 1, 2]   
3    """That was decades ago, Cecil! Anyway, we wan...  [1, 2, 1, 1, 1, 1]   
4    """This Jestxh idiot keeps riding me in dms, t...     [1, 3, 3, 3, 4]   
..                                                 ...                 ...   
621                         women are not dumb bitches        [1, 5, 2, 3]   
622  worst part about getting highlights is sitting...  [1, 3, 1, 1, 1, 1]   
623           yids are not as honest as the rest of us  [5, 5, 4, 5, 4, 1]   
624  you've got to be fucking joking, my room has l...  [3, 3, 1, 1, 2, 1]   
625  {@ClonedDeviant} « probably would have done th...     [1, 1, 1, 2, 1]   

                  toany                           annotatorMinority  \
0          [3, 3, 1, 2]    [{}, pansexual, disabled, trans, {}, {}]   
1    [4, 2, 1, 2, 2, 1]                    [{}, {}, {}, {}, {}, {}]   
2    [2, 1, 1, 1, 1, 3]        [{}, black; female, nan, {}, {}, {}]   
3    [1, 4, 1, 1, 1, 1]       [{}, black; female, nan, {}, {}, nan]   
4       [1, 2, 4, 4, 4]           [{}, NONE, No i do not, none, {}]   
..                  ...                                         ...   
621        [4, 5, 2, 3]  [pansexual, disabled, trans, {}, none, {}]   
622  [1, 5, 1, 1, 1, 1]                [{}, NONE, {}, {}, {}, none]   
623  [5, 5, 4, 5, 4, 1]           [nan, {}, {}, none, {}, bisexual]   
624  [2, 4, 1, 2, 3, 2]                  [{}, {}, {}, {}, none, {}]   
625     [1, 1, 1, 3, 1]                      [{}, {}, {}, {}, none]   

                                                  bert  \
0    [0.0047034588642418385, 0.11112377792596817, 0...   
1    [0.35706740617752075, 0.032704465091228485, 0....   
2    [-0.022276882082223892, 0.17076537013053894, 0...   
3    [0.07474598288536072, 0.24487990140914917, 0.2...   
4    [0.050774428993463516, 0.08187347650527954, 0....   
..                                                 ...   
621  [0.22457970678806305, 0.3840716481208801, -0.4...   
622  [0.1781517118215561, 0.02407737821340561, 0.13...   
623  [0.2983386516571045, 0.31099164485931396, -0.4...   
624  [0.6042274236679077, 0.4851360023021698, -0.24...   
625  [-0.044936660677194595, 0.07898200303316116, 0...   

                      annotatorPolitics      traditionalism  \
0                [-1.0, -1.0, 1.0, 0.5]        [3, 1, 5, 4]   
1    [-0.5, -0.5, 0.5, 0.5, -1.0, -1.0]  [4, 3, 4, 3, 1, 4]   
2    [-0.5, -1.0, 1.0, 0.0, -1.0, -0.5]  [3, 4, 3, 5, 1, 3]   
3     [0.0, -1.0, 1.0, 0.5, -0.5, -1.0]  [4, 4, 3, 4, 5, 5]   
4           [-1.0, 1.0, 0.5, 0.5, -1.0]     [5, 3, 5, 5, 5]   
..                                  ...                 ...   
621               [-1.0, 1.0, 0.5, 0.5]        [1, 5, 5, 5]   
622   [-1.0, 1.0, 0.5, 0.5, -0.5, -0.5]  [5, 3, 5, 2, 4, 3]   
623  [-0.5, -0.5, 0.5, 0.5, -0.5, -0.5]  [3, 5, 2, 5, 4, 3]   
624   [-0.5, -1.0, 0.0, 0.5, 0.0, -0.5]  [4, 5, 5, 3, 5, 5]   
625        [-0.5, -0.5, 0.5, 0.0, -0.5]     [5, 3, 2, 5, 3]   

                                  annotatorRace  \
0                  [black, black, white, white]   
1    [black, black, white, white, white, white]   
2    [black, black, white, white, white, white]   
3    [black, black, white, white, white, white]   
4           [black, black, white, white, white]   
..                                          ...   
621                [black, black, white, white]   
622  [black, black, white, white, white, white]   
623  [black, black, white, white, white, white]   
624  [black, black, white, white, white, white]   
625         [black, white, white, white, white]   

                             annotatorAge  \
0                [25.0, 35.0, 50.0, 25.0]   
1    [40.0, 30.0, 40.0, 30.0, 25.0, 25.0]   
2    [30.0, 40.0, 35.0, 40.0, 25.0, 45.0]   
3    [35.0, 

y=Racism, x={age, race, gender}

In [27]:
def hash(string: str) -> str:
    h = hashlib.new("sha256")
    h.update(string.encode())
    return h.hexdigest()

df = df.loc[:, ["tweet", "racism", "annotatorAge","annotatorRace", "annotatorGender"]]
df.tweet = df.tweet.apply(hash)
df

tweet              racism  \
0    af1eb9224fe451ffd45e139efd44d028d1d77afe466b9e...        [1, 1, 1, 4]   
1    350c15a90cb38d4f42ba41bb5434fdca2b1a7f65c6f5dd...  [1, 1, 5, 5, 1, 1]   
2    1c08d3b6cbb708a287cc433a84f6d7b52ed3ff36f2ce91...  [1, 1, 5, 1, 1, 1]   
3    b7495f0b1706501cc8e22fa035b7d3835a2696861de801...  [1, 1, 5, 5, 1, 1]   
4    bcd9ebe5402b1c7525e2ef436134d886724f6fe85ff7db...     [1, 3, 5, 1, 1]   
..                                                 ...                 ...   
621  e9aade1094bb122fc2a5a8e768c513a470080582cb4139...        [1, 5, 1, 1]   
622  4da726def5d935d6a41b4ab5ca37e93758a6566c00b04f...  [1, 5, 2, 3, 2, 1]   
623  41228ab2450e49fbdd98525d7fbd017b2c1a3b0b2f6b94...  [1, 1, 4, 1, 2, 1]   
624  cc57457f38462bf3e1dd6dbef60f8f90f458cd3c48cdc2...  [1, 4, 1, 3, 3, 1]   
625  5f01529d67d1e2dc26aef17fecc7c8b5071c9eacbbdac3...     [1, 1, 4, 3, 1]   

                             annotatorAge  \
0                [25.0, 35.0, 50.0, 25.0]   
1    [40.0, 30.0, 40.0, 30.0, 25.0, 25.0]   
2    [30.0, 40.0, 35.0, 40.0, 25.0, 45.0]   
3    [35.0, 40.0, 35.0, 40.0, 35.0, 45.0]   
4          [45.0, 45.0, 45.0, 60.0, 40.0]   
..                                    ...   
621              [35.0, 30.0, 45.0, 65.0]   
622  [45.0, 45.0, 35.0, 35.0, 25.0, 50.0]   
623  [30.0, 30.0, 40.0, 60.0, 35.0, 40.0]   
624  [40.0, 25.0, 40.0, 35.0, 45.0, 35.0]   
625        [30.0, 30.0, 40.0, 30.0, 50.0]   

                                  annotatorRace  \
0                  [black, black, white, white]   
1    [black, black, white, white, white, white]   
2    [black, black, white, white, white, white]   
3    [black, black, white, white, white, white]   
4           [black, black, white, white, white]   
..                                          ...   
621                [black, black, white, white]   
622  [black, black, white, white, white, white]   
623  [black, black, white, white, white, white]   
624  [black, black, white, white, white, white]   
625         [black, white, white, white, white]   

                                annotatorGender  
0                      [man, man, woman, woman]  
1          [woman, woman, woman, man, man, man]  
2        [woman, woman, man, woman, man, woman]  
3        [man, woman, man, woman, woman, woman]  
4               [man, man, woman, woman, woman]  
..                                          ...  
621                    [man, woman, man, woman]  
622            [man, man, woman, man, man, man]  
623        [man, man, man, woman, woman, woman]  
624  [woman, woman, woman, woman, woman, woman]  
625                   [man, man, man, man, man]  

[626 rows x 5 columns]

In [28]:
def find_inconsistent_rows(df):
    inconsistent_rows = []
    
    for index, row in df.iterrows():
        list_lengths = [len(row[col]) for col in df.columns if isinstance(row[col], list)]
        
        if len(set(list_lengths)) > 1:  # Check if lengths are inconsistent
            inconsistent_rows.append(index)
    
    return df.loc[inconsistent_rows]


find_inconsistent_rows(df)

Empty DataFrame
Columns: [tweet, racism, annotatorAge, annotatorRace, annotatorGender]
Index: []

## Annotator analysis

### Gender

In [29]:
def extract_annotations_and_attributes(
    df: pd.DataFrame, annot_col: str, attribute_col: str, key_col: str
) -> tuple[list, list]:
    all_annotations = []
    all_attributes = []
    all_keys = []

    for _, row in df.iterrows():
        all_annotations.extend(row[annot_col])
        all_attributes.extend(row[attribute_col])
        # extend the key for each value in the above extracted list
        all_keys.extend([row[key_col]] * len(row[annot_col]))

    return all_annotations, all_attributes, all_keys

Seems to be better to include all ap-stats instead of selecting the max.

When using max, all values are always statistically significant. 

In [30]:
for explanatory_var in ["annotatorAge", "annotatorRace", "annotatorGender"]:
    annotations, attributes, keys = extract_annotations_and_attributes(
        df=df, annot_col="racism", attribute_col=explanatory_var, key_col="tweet"
    )
    stat = aposteriori.aposteriori_unimodality(
        annotations=annotations, annotator_group=attributes, comment_group=keys
    )
    print(f"{explanatory_var}: {stat:0.5f}")

DEBUG: final stats:  DescribeResult(nobs=626, minmax=(-1.0, 1.0), mean=0.15933997667224503, variance=0.0688951034655964, skewness=0.32081317077915966, kurtosis=1.1419200291184124)
annotatorAge: 0.00000
DEBUG: final stats:  DescribeResult(nobs=626, minmax=(-1.0, 1.0), mean=-0.07362886048988287, variance=0.07180378363507277, skewness=0.12910486238001834, kurtosis=1.2152414864920038)
annotatorRace: 0.99988
DEBUG: final stats:  DescribeResult(nobs=626, minmax=(-1.0, 1.0), mean=-0.01645367412140577, variance=0.062164324931960724, skewness=0.3258335221550713, kurtosis=1.1017718253320181)
annotatorGender: 0.73001
